In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import colorcet as cc
import matplotlib.pyplot as plt
from geopy.distance import distance

In [5]:
# load data
df = pd.read_csv('merge.csv')

# city lat/lng found from: https://simplemaps.com/data/us-cities
citi = pd.read_csv('uscities.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  int64  
 4   arrival_city                         18451 non-null  int64  
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [6]:
citi = citi[['city', 'lat', 'lng']]

citi.head()

,city,lat,lng
0,New York,40.6943,-73.9249
1,Los Angeles,34.1141,-118.4068
2,Chicago,41.8375,-87.6866
3,Miami,25.7840,-80.2101
4,Houston,29.7860,-95.3885


In [7]:
df.head()

,Year,Quarter,Total_Mkt_Fare,depart_city,arrival_city,airlineID,Passengers_by_Carrier,CarriersMktShare,CarrierAvgFare,Carrier_MinFareIncrement,...,arrival_state,Avg_TDOMT_COST_carrier,avg_TDOMT_GALLONS_carrier,Monthly Average Temperature_depart,Monthly Total Precepitation_depart,Monthly Average Windspeed_depart,Monthly Average Temperature_arrival,Monthly Total Precepitation_arrival,Monthly Average Windspeed_arrival,distance
0,2006,3,87.41,93,112,19393,34330,0.59,80.44,75,...,41,1.871066e+08,1.195102e+08,20.300000,12.333333,10.866667,20.033333,5.666667,12.466667,287.996148
1,2006,3,87.41,93,112,19393,34330,0.59,80.44,75,...,41,1.871066e+08,1.195102e+08,20.300000,12.333333,10.866667,20.166667,5.666667,7.466667,287.996148
2,2006,3,87.41,93,112,19393,34330,0.59,80.44,75,...,41,1.871066e+08,1.195102e+08,20.300000,12.333333,10.866667,20.033333,5.666667,12.466667,287.996148
3,1997,4,224.47,16,92,20355,68280,0.92,226.21,75,...,34,4.954367e+07,8.412267e+07,9.721248,68.756014,12.198811,5.266667,69.666667,12.033333,478.632113
4,2003,1,234.32,46,123,20355,11770,0.98,234.92,100,...,7,4.883800e+07,5.535633e+07,6.966667,107.000000,10.533333,2.900000,102.000000,15.133333,247.783098


In [8]:
# get all cities in the dataset
city1 = df.depart_city.unique()
city2 = df.arrival_city.unique()
cities = list(np.unique(np.append(city1, city2)))

len(cities)

125

In [9]:
[i for i in cities if i not in citi.city.unique()]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125]

In [12]:
import numpy as np

# Check and handle missing or non-string values
df['depart_city'] = df['depart_city'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x)
df['arrival_city'] = df['arrival_city'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x)

# Update rows with 'New York City'
row_index1 = df.loc[df['depart_city'] == 'New York City'].index
row_index2 = df.loc[df['arrival_city'] == 'New York City'].index
df.loc[row_index1, 'depart_city'] = 'New York'
df.loc[row_index2, 'arrival_city'] = 'New York'

# Get all cities in the dataset
city1 = df.depart_city.unique()
city2 = df.arrival_city.unique()
cities = list(np.unique(np.append(city1, city2)))

len(cities)


125

In [13]:
[i for i in cities if i not in citi.city.unique()]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125]

In [14]:
df = df[~(df.depart_city == 'Nantucket')]

# get all cities in the dataset
city1 = df.depart_city.unique()
city2 = df.arrival_city.unique()
cities = list(np.unique(np.append(city1, city2)))

[i for i in cities if i not in citi.city.unique()]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125]

In [16]:
from math import radians, sin, cos, sqrt, atan2

# Define a function to calculate the distance between two points using the Haversine formula
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 3958.8  # Radius of the Earth in miles

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

# Define a function to calculate the distance between two states
def calculate_distance(row, citi_df):
    citi1_info = citi_df[citi_df['city'] == row['depart_city']]
    citi2_info = citi_df[citi_df['city'] == row['arrival_city']]

    # Check if indices are not empty before accessing the information
    if not citi1_info.empty and not citi2_info.empty:
        lat1, lon1 = citi1_info.iloc[0]['lat'], citi1_info.iloc[0]['lng']
        lat2, lon2 = citi2_info.iloc[0]['lat'], citi2_info.iloc[0]['lng']

        return haversine_distance(lat1, lon1, lat2, lon2)
    else:
        # Handle the case where information is not found
        return None

# Apply the function to calculate distances for each row in the DataFrame
df['distance'] = df.apply(lambda row: calculate_distance(row, citi), axis=1)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  object 
 4   arrival_city                         18451 non-null  object 
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [18]:
keep = ['Year', 'Quarter', 'Total_Mkt_Fare', 'depart_city', 'arrival_city', 'airlineID',
        'Passengers_by_Carrier', 'CarriersMktShare', 'CarrierAvgFare', 'Carrier_MinFareIncrement',
        'CarrierMinPassangerShare', 'CarrierMaxFareIncrement', 'CarrierMaxPassengerShare',
        'depart_state', 'arrival_state', 'Avg_TDOMT_COST_carrier', 'avg_TDOMT_GALLONS_carrier',
        'Monthly Average Temperature_depart', 'Monthly Total Precepitation_depart',
        'Monthly Average Windspeed_depart', 'Monthly Average Temperature_arrival',
        'Monthly Total Precepitation_arrival', 'Monthly Average Windspeed_arrival', 'distance']
df = df[keep]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  object 
 4   arrival_city                         18451 non-null  object 
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [19]:
# get all cities in the dataset
state1 = df.depart_state.unique()
state2 = df.arrival_state.unique()
states = list(np.unique(np.append(state1, state2)))

In [20]:
citi_dict = {x: index for index, x in enumerate(cities, start=1)}
# extract coded dataset
df = df.replace({'depart_city': citi_dict, 'arrival_city': citi_dict})

state_dict = {x: index for index, x in enumerate(states, start=1)}
# extract coded dataset
df = df.replace({'depart_state': state_dict, 'arrival_state': state_dict})



df.head()

,Year,Quarter,Total_Mkt_Fare,depart_city,arrival_city,airlineID,Passengers_by_Carrier,CarriersMktShare,CarrierAvgFare,Carrier_MinFareIncrement,...,arrival_state,Avg_TDOMT_COST_carrier,avg_TDOMT_GALLONS_carrier,Monthly Average Temperature_depart,Monthly Total Precepitation_depart,Monthly Average Windspeed_depart,Monthly Average Temperature_arrival,Monthly Total Precepitation_arrival,Monthly Average Windspeed_arrival,distance
0,2006,3,87.41,93,112,19393,34330,0.59,80.44,75,...,41,1.871066e+08,1.195102e+08,20.300000,12.333333,10.866667,20.033333,5.666667,12.466667,None
1,2006,3,87.41,93,112,19393,34330,0.59,80.44,75,...,41,1.871066e+08,1.195102e+08,20.300000,12.333333,10.866667,20.166667,5.666667,7.466667,None
2,2006,3,87.41,93,112,19393,34330,0.59,80.44,75,...,41,1.871066e+08,1.195102e+08,20.300000,12.333333,10.866667,20.033333,5.666667,12.466667,None
3,1997,4,224.47,16,92,20355,68280,0.92,226.21,75,...,34,4.954367e+07,8.412267e+07,9.721248,68.756014,12.198811,5.266667,69.666667,12.033333,None
4,2003,1,234.32,46,123,20355,11770,0.98,234.92,100,...,7,4.883800e+07,5.535633e+07,6.966667,107.000000,10.533333,2.900000,102.000000,15.133333,None


In [21]:
df['Monthly Average Temperature_depart'] = df.groupby(['Year', 'Quarter', 'depart_city']) \
    [['Monthly Average Temperature_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_depart'] = df.groupby(['Year', 'Quarter', 'depart_city']) \
    [['Monthly Total Precepitation_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_depart'] = df.groupby(['Year', 'Quarter', 'depart_city']) \
    [['Monthly Average Windspeed_depart']].transform(lambda x: x.fillna(x.mean()))

df['Monthly Average Temperature_arrival'] = df.groupby(['Year', 'Quarter', 'arrival_city']) \
    [['Monthly Average Temperature_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_arrival'] = df.groupby(['Year', 'Quarter', 'arrival_city']) \
    [['Monthly Total Precepitation_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_arrival'] = df.groupby(['Year', 'Quarter', 'arrival_city']) \
    [['Monthly Average Windspeed_arrival']].transform(lambda x: x.fillna(x.mean()))

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  int64  
 4   arrival_city                         18451 non-null  int64  
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [23]:
df['Monthly Average Temperature_depart'] = df.groupby(['Quarter', 'depart_city']) \
    [['Monthly Average Temperature_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_depart'] = df.groupby(['Quarter', 'depart_city']) \
    [['Monthly Total Precepitation_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_depart'] = df.groupby(['Quarter', 'depart_city']) \
    [['Monthly Average Windspeed_depart']].transform(lambda x: x.fillna(x.mean()))

df['Monthly Average Temperature_arrival'] = df.groupby(['Quarter', 'arrival_city']) \
    [['Monthly Average Temperature_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_arrival'] = df.groupby(['Quarter', 'arrival_city']) \
    [['Monthly Total Precepitation_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_arrival'] = df.groupby(['Quarter', 'arrival_city']) \
    [['Monthly Average Windspeed_arrival']].transform(lambda x: x.fillna(x.mean()))

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  int64  
 4   arrival_city                         18451 non-null  int64  
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [25]:
df['Monthly Average Temperature_depart'] = df.groupby(['Year', 'Quarter', 'depart_state']) \
    [['Monthly Average Temperature_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_depart'] = df.groupby(['Year', 'Quarter', 'depart_state']) \
    [['Monthly Total Precepitation_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_depart'] = df.groupby(['Year', 'Quarter', 'depart_state']) \
    [['Monthly Average Windspeed_depart']].transform(lambda x: x.fillna(x.mean()))

df['Monthly Average Temperature_arrival'] = df.groupby(['Year', 'Quarter', 'arrival_state']) \
    [['Monthly Average Temperature_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_arrival'] = df.groupby(['Year', 'Quarter', 'arrival_state']) \
    [['Monthly Total Precepitation_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_arrival'] = df.groupby(['Year', 'Quarter', 'arrival_state']) \
    [['Monthly Average Windspeed_arrival']].transform(lambda x: x.fillna(x.mean()))

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  int64  
 4   arrival_city                         18451 non-null  int64  
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [27]:
df['Monthly Average Temperature_depart'] = df.groupby(['Quarter', 'depart_state']) \
    [['Monthly Average Temperature_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_depart'] = df.groupby(['Quarter', 'depart_state']) \
    [['Monthly Total Precepitation_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_depart'] = df.groupby(['Quarter', 'depart_state']) \
    [['Monthly Average Windspeed_depart']].transform(lambda x: x.fillna(x.mean()))

df['Monthly Average Temperature_arrival'] = df.groupby(['Quarter', 'arrival_state']) \
    [['Monthly Average Temperature_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_arrival'] = df.groupby(['Quarter', 'arrival_state']) \
    [['Monthly Total Precepitation_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_arrival'] = df.groupby(['Quarter', 'arrival_state']) \
    [['Monthly Average Windspeed_arrival']].transform(lambda x: x.fillna(x.mean()))

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  int64  
 4   arrival_city                         18451 non-null  int64  
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [ ]:
df['Monthly Average Temperature_depart'] = df.groupby(['Year', 'Quarter']) \
    [['Monthly Average Temperature_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_depart'] = df.groupby(['Year', 'Quarter']) \
    [['Monthly Total Precepitation_depart']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_depart'] = df.groupby(['Year', 'Quarter']) \
    [['Monthly Average Windspeed_depart']].transform(lambda x: x.fillna(x.mean()))
/
df['Monthly Average Temperature_arrival'] = df.groupby(['Year', 'Quarter']) \
    [['Monthly Average Temperature_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Total Precepitation_arrival'] = df.groupby(['Year', 'Quarter']) \
    [['Monthly Total Precepitation_arrival']].transform(lambda x: x.fillna(x.mean()))
df['Monthly Average Windspeed_arrival'] = df.groupby(['Year', 'Quarter']) \
    [['Monthly Average Windspeed_arrival']].transform(lambda x: x.fillna(x.mean()))

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18451 entries, 0 to 18450
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 18451 non-null  int64  
 1   Quarter                              18451 non-null  int64  
 2   Total_Mkt_Fare                       18451 non-null  float64
 3   depart_city                          18451 non-null  int64  
 4   arrival_city                         18451 non-null  int64  
 5   airlineID                            18451 non-null  int64  
 6   Passengers_by_Carrier                18451 non-null  int64  
 7   CarriersMktShare                     18451 non-null  float64
 8   CarrierAvgFare                       18451 non-null  float64
 9   Carrier_MinFareIncrement             18451 non-null  int64  
 10  CarrierMinPassangerShare             18451 non-null  float64
 11  CarrierMaxFareIncrement     

In [30]:
df.to_csv('merge.csv', index=False)

In [31]:
from sklearn.model_selection import train_test_split

# make 30% test set and 70% training
X = df.loc[:, df.columns != 'Total_Mkt_Fare']
y = df.Total_Mkt_Fare
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# devide test into half to form validation set: 15% test, 15% valid, 70% train
X_hold, X_valid, y_hold, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=21)

In [32]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_valid.shape, y_valid.shape

((12915, 23), (12915,), (5536, 23), (5536,), (2768, 23), (2768,))

In [33]:
# functions to calculate accuracy
def RMSE(y, ypred):
    differences = [(x-y)**2 for x, y in zip(ypred, y)]

    return np.sqrt(sum(differences) / len(differences))

def MAE(y, ypred):
    return abs(y-ypred).mean()

def get_acc(y, ypred):
    correct = []

    for i in range(len(y)):
        correct.append(y[i]==ypred[i])

    return sum(correct) / len(correct)

## **Random Forest Model**

**Random forest model on the validation set**`



In [50]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Instantiate the imputer with a strategy (e.g., mean, median, or constant)
imputer = SimpleImputer(strategy='mean')

# Fit and transform the imputer on the training data
X_train_imputed = imputer.fit_transform(X_train)
X_valid_imputed = imputer.transform(X_valid)

# Instantiate the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the imputed training data
rf_model.fit(X_train_imputed, y_train)

# Predictions on the imputed validation set
rf_preds_valid = rf_model.predict(X_valid_imputed)

# Evaluate the model on the validation set
rmse_valid = np.sqrt(mean_squared_error(y_valid, rf_preds_valid))
mae_valid = mean_absolute_error(y_valid, rf_preds_valid)

print(f'Random Forest Validation RMSE: {rmse_valid}, MAE: {mae_valid}')


Random Forest Validation RMSE: 13.646216386188303, MAE: 6.114488619942192


**Random forest model on the test set**`





In [56]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Instantiate the imputer with a strategy (e.g., mean, median, or constant)
imputer = SimpleImputer(strategy='mean')

# Fit and transform the imputer on the training data
X_train_imputed = imputer.fit_transform(X_train)
X_valid_imputed = imputer.transform(X_valid)

# Instantiate the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Training the model on the imputed training data
rf_model.fit(X_train_imputed, y_train)

# Predictions on the imputed test set
X_test_imputed = imputer.transform(X_test)
rf_preds_test = rf_model.predict(X_test_imputed)

# Evaluate the model on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, rf_preds_test))
mae_test = mean_absolute_error(y_test, rf_preds_test)

print(f'Random Forest Test RMSE: {rmse_test}, MAE: {mae_test}')


Random Forest Test RMSE: 13.171162680625898, MAE: 6.068562138728323


**Calculating the R^2 for the Random Forest Model**

In [57]:
from sklearn.metrics import r2_score

# Predictions on the imputed validation set
rf_preds_valid = rf_model.predict(X_valid_imputed)

# Calculating R² on the validation set
r2_valid = r2_score(y_valid, rf_preds_valid)

print(f'Random Forest Validation R²: {r2_valid}')


Random Forest Validation R²: 0.9777891456092935


**Calculating Random forest Cross-Validation Mean R^2**

In [58]:
from sklearn.model_selection import cross_val_score

# Use cross_val_score to perform cross-validation
# rf_model is the RandomForestRegressor instance
# X_train_imputed is the imputed training data
# y_train is the training labels
# cv is the number of folds for cross-validation

# Replacing rf_model, X_train_imputed, y_train, and cv with the actual model and data
cv_scores = cross_val_score(rf_model, X_train_imputed, y_train, cv=5, scoring='r2')

# Calculating the mean R² across cross-validation folds
mean_r2_cv = np.mean(cv_scores)

print(f'Random Forest Cross-Validation Mean R²: {mean_r2_cv}')


Random Forest Cross-Validation Mean R²: 0.9782346086868193
